# M2608.001300 Machine Learning<br> Assignment #5 Final Projects (Pytorch)


Copyright (C) Data Science & AI Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them.

**For understanding of this work, please carefully look at given PPT file.**

Note: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.

In [1]:
from google.colab import drive
drive.mount('/content/drive')


import os
import random

import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
from PIL import Image
import resnet

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import natsort

Load datasets


In [3]:
NUMBER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
ALPHABET = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
NONE = ['NONE'] # label for empty space
ALL_CHAR_SET = NUMBER + ALPHABET + NONE
ALL_CHAR_SET_LEN = len(ALL_CHAR_SET)
MAX_CAPTCHA = 7

def encode(a):
    onehot = [0]*ALL_CHAR_SET_LEN
    idx = ALL_CHAR_SET.index(a)
    onehot[idx] += 1
    return onehot

# modified dataset class
class Mydataset(Dataset):
    def __init__(self, img_path, label_path, is_train=True, transform=None):
        self.path = img_path
        self.label_path = label_path
        if is_train: 
            self.img = os.listdir(self.path)[:10000]
           # self.img= natsort.natsorted(self.img)
            self.labels = open(self.label_path, 'r').read().split('\n')[:-1][:10000]
        else: 
            self.img = os.listdir(self.path)[:1000]
           # self.img= natsort.natsorted(self.img)
            self.labels = open(self.label_path, 'r').read().split('\n')[:-1][:1000]
        
        self.transform = transform
        self.max_length = MAX_CAPTCHA
        
    def __getitem__(self, idx):
        img_path = self.img[idx]
        img = Image.open(f'{self.path}/{self.img[idx]}')
        img = img.convert('L')
        label = self.labels[idx]
        label_oh = []
        # one-hot for each character
        for i in range(self.max_length):
            if i < len(label):
                label_oh += encode(label[i])
            else:
                #label_oh += [0]*ALL_CHAR_SET_LEN
                label_oh += encode('NONE')
            
        if self.transform is not None:
            img = self.transform(img)
        return img, np.array(label_oh), label
    
    def __len__(self):
        return len(self.img)

transform = transforms.Compose([
    transforms.Resize([160, 60]),
    transforms.ToTensor(),
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################

    transforms.Normalize((0.894832,),(0.109865,)),#mean variance 로 normalize
##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################
])



In [5]:
"""Loading DATA"""
# Change to your own data foler path!
gPath = '/content/drive/My Drive/final_project/'

train_ds = Mydataset(gPath+'Data/train/', gPath+'Data/train.txt',transform=transform)# data -> Data 고쳐주기 
test_ds = Mydataset(gPath+'Data/test/', gPath+'Data/test.txt',False, transform)# Data -> data 고쳐주기 
train_dl = DataLoader(train_ds, batch_size=128, num_workers=4)
test_dl = DataLoader(test_ds, batch_size=1, num_workers=4)

In [6]:
"""To CUDA for local run"""
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#GPUID = '4' # define GPUID
#os.environ["CUDA_VISIBLE_DEVICES"] = str(GPUID)


Problem 1: Design LSTM model for catcha image recognition. (10 points)

In [9]:
teacher_forcing_ratio = 0.4

class LSTM(nn.Module):
    def __init__(self, cnn_dim, hidden_size, vocab_size, num_layers=1):
        super(LSTM, self).__init__()
        
        # define the properties
        self.cnn_dim = cnn_dim
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        
        # lstm cell
        self.lstm_cell = nn.LSTMCell(input_size=self.vocab_size, hidden_size=hidden_size)
    
        # output fully connected layer
        self.fc_in = nn.Linear(in_features=self.cnn_dim, out_features=self.vocab_size)
        self.fc_out = nn.Linear(in_features=self.hidden_size, out_features=self.vocab_size)
    
        # embedding layer
        self.embed = nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=self.vocab_size)
    
        # activations
        self.softmax = nn.Softmax(dim=1)
      


    
    def forward(self, features, captions, flagship):

        batch_size = features.size(0)
        cnn_dim = features.size(1)
    
        hidden_state = torch.zeros((batch_size, self.hidden_size)).cuda()
        cell_state = torch.zeros((batch_size, self.hidden_size)).cuda()
    
        # define the output tensor placeholder
        outputs = torch.empty((batch_size, captions.size(1), self.vocab_size)).cuda()

        # embed the captions
        captions_embed = self.embed(captions)

##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################
       
      #  outputs[:,0,:]=self.fc_out(hidden_state)
     #   output_index = outputs[:,0,:].argmax(axis=1)
        if flagship == "train":
          use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
          #outputs[:,0,:]=self.fc_out(hidden_state)
          if use_teacher_forcing:
            for i in range(0,7):
              if i == 0:
                hidden_state, cell_state = self.lstm_cell((self.fc_in(features)),(hidden_state,cell_state))
              else:
                hidden_state, cell_state = self.lstm_cell(captions_embed[:,i-1,:], (hidden_state, cell_state))
              outputs[:,i,:] = (self.fc_out(hidden_state))
          else:
            for i in range(0,7):
              if i == 0:
                hidden_state, cell_state = self.lstm_cell((self.fc_in(features)),(hidden_state,cell_state))
              else:
                hidden_state, cell_state = self.lstm_cell(self.embed(torch.argmax(outputs[:,i-1,:],1)), (hidden_state, cell_state))
              outputs[:,i,:] = (self.fc_out(hidden_state))
          outputs = self.softmax(outputs)
        else:
            for i in range(0,7):
                if i == 0:
                  hidden_state, cell_state = self.lstm_cell((self.fc_in(features)),(hidden_state,cell_state))     
                else:
                  hidden_state, cell_state = self.lstm_cell(self.embed(torch.argmax(outputs[:,i-1,:],1)), (hidden_state, cell_state))
            outputs[:,i,:] = (self.fc_out(hidden_state))     
        

##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################
        return outputs 




Problem 2: 

*   1.Connect CNN model to the desinged LSTM model.
*   2.Replace ResNet to your own CNN model from Assignment3.


          


1. resnet 연결, cnn 연결 시 각 주석 부분을 바꿔서 처리! 
2. 즉 resnet 연결시 cnn 연결파트 주석처리
3. cnn 연결시 resnet 연결파트 주석처리

In [10]:
class Inception(nn.Module):
    def __init__(self, in_planes, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_planes):
        super(Inception, self).__init__()
        # 1x1 conv branch
        self.b1 = nn.Sequential(
            nn.Conv2d(in_planes, n1x1, kernel_size=1),
            nn.BatchNorm2d(n1x1),
            nn.ReLU(True),
        )

        # 1x1 conv -> 3x3 conv branch
        self.b2 = nn.Sequential(
            nn.Conv2d(in_planes, n3x3red, kernel_size=1),
            nn.BatchNorm2d(n3x3red),
            nn.ReLU(True),
            nn.Conv2d(n3x3red, n3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(n3x3),
            nn.ReLU(True),
        )

        # 1x1 conv -> 5x5 conv branch
        self.b3 = nn.Sequential(
            nn.Conv2d(in_planes, n5x5red, kernel_size=1),
            nn.BatchNorm2d(n5x5red),
            nn.ReLU(True),
            nn.Conv2d(n5x5red, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
            nn.Conv2d(n5x5, n5x5, kernel_size=5, padding=2),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
        )

        # 3x3 pool -> 1x1 conv branch
        self.b4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_planes, pool_planes, kernel_size=1),
            nn.BatchNorm2d(pool_planes),
            nn.ReLU(True),
        )

    def forward(self, x):
        y1 = self.b1(x)
        y2 = self.b2(x)
        y3 = self.b3(x)
        y4 = self.b4(x)
        return torch.cat([y1,y2,y3,y4], 1)
# Define a CNN model
class BetterNet(nn.Module):
    def __init__(self):
        super(BetterNet, self).__init__()
        ##############################################################################
        #                          IMPLEMENT YOUR CODE                               #
        ##############################################################################
        self.firstlayers = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=5, padding=2),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
        )
        self.x1 = Inception(64,  56,  40, 32, 60, 32, 8)
        self.maxpool = nn.MaxPool2d(2,2)
        self.x3 = Inception(128,  56,  40, 32, 60, 32, 8)
        self.x9 = Inception(128,  200,  160, 140, 240, 120, 52)
        self.linear = nn.Linear(512*5, 512)
        ##############################################################################
        #                          END OF YOUR CODE                                  #
        ##############################################################################

    def forward(self, x):
        ##############################################################################
        #                          IMPLEMENT YOUR CODE                               #
        ##############################################################################
        x = self.firstlayers(x)
        x= self.maxpool(x)
        x = self.x1(x)
        x = self.maxpool(x)
        x = self.x3(x)
        x = self.maxpool(x)
        x = self.x9(x)
        x = self.maxpool(x)
        x = x.view(x.size(0),-1)
        out = self.linear(x)
        ##############################################################################
        #                          END OF YOUR CODE                                  #
        ##############################################################################
        return out

In [11]:
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################
"""ResNet
betternet = resnet.resnet18(pretrained=False)
betternet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
betternet.fc = nn.Linear(in_features=512, out_features=ALL_CHAR_SET_LEN*MAX_CAPTCHA, bias=True)
betternet = betternet.to(device)"""

#cnn
betternet = BetterNet()
betternet = betternet.to(device)
##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################

       
# LSTM
cnn_dim=512 #resnet18-512
hidden_size=8
vocab_size=37 #ALL_CHAR_SET_LEN
lstm = LSTM(cnn_dim=cnn_dim, hidden_size=hidden_size, vocab_size=vocab_size)
lstm = lstm.to(device)

# loss, optimizer
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################
loss_func = nn.MultiLabelSoftMarginLoss()
cnn_optim = torch.optim.Adam(betternet.parameters(), lr=0.001)
lstm_optim = torch.optim.Adam(lstm.parameters(),lr = 0.001)
##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################

Problem3: Find hyper-parameters.


find hyper parameters:
1. learning rate
2. loss function 종류
3. teacher forcing 할 때 random 기준 값
4. cnn 모델의 선택
5. batchsize , fc_in, fc_out shape 개수
6. output에서 softmax 사용 여부

In [12]:
def get_char_count(arg1):
    c0 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c5 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*5:ALL_CHAR_SET_LEN*6])]
    c6 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*6:ALL_CHAR_SET_LEN*7])]
    return c0, c1, c2,c3, c4, c5, c6 
"""TRAINING"""
print_interval = 15
max_epoch = 10

for epoch in range(max_epoch):
    for step, i in enumerate(train_dl):
        img, label_oh, label = i
        img = Variable(img).cuda()
        label_oh = Variable(label_oh.float()).cuda()
        batch_size, _ = label_oh.shape
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################
        label_oh1 = torch.reshape(label_oh, (batch_size, 7, 37))#label oh 처리를 위해 코드 추가
        label_oh2 = torch.argmax(label_oh1, dim=2)#label oh 처리를 위해 코드 추가

        '''resnet 사용
        pred, feature = betternet(img)#cnn lstm 연결 위해 코드 추가
        pred1 = lstm(torch.flatten(betternet.avgpool(feature),1),label_oh2,"train")'''#cnn lstm 연결 위해 코드 추가
        
        '''cnn 사용'''
        feature = betternet(img)#cnn lstm 연결 위해 코드 추가
        pred1 = lstm(feature,label_oh2, "train")#cnn lstm 연결 위해 코드 추가
        loss = loss_func(pred1, label_oh1)#label oh 처리를 위해 코드 추가
        cnn_optim.zero_grad()
        lstm_optim.zero_grad() #추가
        loss.backward()
        lstm_optim.step() #추가
        cnn_optim.step()
##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################
        if (step+1)%print_interval == 0:
          ## train 시키면서 값 비교하기 위해 코드 추가 
          pred_captions = torch.argmax(pred1, axis=2) 
          ctr = []
          dtr = []
          for i in range(7): 
            ctr.append(ALL_CHAR_SET[label_oh2[0][i]])
            dtr.append(ALL_CHAR_SET[pred_captions[0][i]])
          print('c_train : ', ctr[0],ctr[1],ctr[2],ctr[3],ctr[4],ctr[5],ctr[6])
          print('d_train : ', dtr[0],dtr[1],dtr[2],dtr[3],dtr[4],dtr[5],dtr[6])
          print('epoch:', epoch+1, 'step:', step+1, 'loss:', loss.item())
          print("\t")    

print("finish training")
torch.save(betternet.state_dict(),"./betternet.pth")           
torch.save(lstm.state_dict(),"./LSTMcell.pth")   
print("saved model")

c_train :  n t l 8 NONE NONE NONE
d_train :  6 1 1 y p NONE o
epoch: 1 step: 15 loss: 0.762991726398468
	
c_train :  s 5 NONE NONE NONE NONE NONE
d_train :  b w y h NONE NONE NONE
epoch: 1 step: 30 loss: 0.7628993391990662
	
c_train :  d e NONE NONE NONE NONE NONE
d_train :  b 1 y 8 NONE NONE NONE
epoch: 1 step: 45 loss: 0.7627525329589844
	
c_train :  g r p NONE NONE NONE NONE
d_train :  b m o i i NONE NONE
epoch: 1 step: 60 loss: 0.7627040147781372
	
c_train :  1 5 f p k NONE NONE
d_train :  b m o i i NONE NONE
epoch: 1 step: 75 loss: 0.7624768018722534
	
c_train :  n t l 8 NONE NONE NONE
d_train :  b m m h h NONE NONE
epoch: 2 step: 15 loss: 0.762397289276123
	
c_train :  s 5 NONE NONE NONE NONE NONE
d_train :  b b 5 i NONE NONE NONE
epoch: 2 step: 30 loss: 0.7623698711395264
	
c_train :  d e NONE NONE NONE NONE NONE
d_train :  4 b 5 1 p NONE NONE
epoch: 2 step: 45 loss: 0.7621845602989197
	
c_train :  g r p NONE NONE NONE NONE
d_train :  4 5 5 u p NONE NONE
epoch: 2 step: 60 loss: 

In [14]:
"""TEST"""
def get_char_count(arg1):
    c0 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c5 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*5:ALL_CHAR_SET_LEN*6])]
    c6 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*6:ALL_CHAR_SET_LEN*7])]
    return c0, c1, c2,c3, c4, c5, c6 
 


char_correct = 0
word_correct = 0
total = 0

betternet.eval()
lstm.eval()

with torch.no_grad():
    for step, (img, label_oh, label) in enumerate(test_dl):
        char_count =0
        img = Variable(img).cuda()
        label_oh = Variable(label_oh.float()).cuda()
        batch_size, _ = label_oh.shape
        label_oh1 = torch.reshape(label_oh, (batch_size, 7, 37)) #추가 label oh shape 처리 
        label_oh2 = torch.argmax(label_oh1, dim=2) #추가 label oh shape 처리 
        
        '''resnet 사용
        pred_resnet, feature = betternet(img) #추가 lstm cnn 연결 
        pred = lstm(torch.flatten(betternet.avgpool(feature),1),label_oh2,"test")'''#추가 lstm cnn 연결 
     
        #cnn 사용
        pred_cnn = betternet(img)#추가 lstm cnn 연결 
        pred = lstm(pred_cnn,label_oh2,"test")#추가 lstm cnn 연결 
  
        pred = torch.reshape(pred, (1,7*37))#추가 label oh shape 처리  

        label_len = label[0]
        pred = pred.squeeze(0)
        label_oh = label_oh.squeeze(0)
      
        c0,c1,c2,c3,c4,c5,c6 = get_char_count(pred.squeeze()) 
        d0,d1,d2,d3,d4,d5,d6 = get_char_count(label_oh) 
         
      
        c = '%s%s%s%s%s%s%s' % (c0, c1, c2, c3, c4, c5, c6)
        d = '%s%s%s%s%s%s%s' % (d0, d1, d2, d3, d4, d5, d6)
        print(c)
        print(d)
        print("\t")
    
        char_count += (c0==d0)+(c1==d1)+(c2==d2)+(c3==d3)+(c4==d4)+(c5==d5)+(c6==d6)
        char_correct += char_count

        if(bool(str(label[0]) in str(c))):
            word_correct+=1

        total += 1
       
print(100/7*char_correct/total)
print(100*word_correct/total)
"""END TEST"""

b9xNONENONENONENONE
b9xNONENONENONENONE
	
mbNONENONENONENONENONE
mbNONENONENONENONENONE
	
d5q7qhNONE
d5q7qhNONE
	
6tl0kqNONE
6tl0kqv
	
t1NONENONENONENONENONE
t1NONENONENONENONENONE
	
avhjn3NONE
avhjn3z
	
74z0zNONENONE
74z0zNONENONE
	
f1kfaNONENONE
f1kfaNONENONE
	
sripnsNONE
sripnsNONE
	
bg4NONENONENONENONE
bg4NONENONENONENONE
	
gmb45tNONE
gmb45tz
	
sr5NONENONENONENONE
sr5NONENONENONENONE
	
0ntNONENONENONENONE
0ntNONENONENONENONE
	
lxfg98NONE
lxfg98NONE
	
2b8oNONENONENONE
2b8oNONENONENONE
	
kr25NONENONENONE
kr25NONENONENONE
	
flNONENONENONENONENONE
flNONENONENONENONENONE
	
0tiwrdNONE
0tiwrdNONE
	
k5NONENONENONENONENONE
k5NONENONENONENONENONE
	
8kNONENONENONENONENONE
8kNONENONENONENONENONE
	
gginNONENONENONE
gginNONENONENONE
	
qc6eNONENONENONE
qc6eNONENONENONE
	
giz6rvNONE
giz6rvNONE
	
tf15NONENONENONE
tf15NONENONENONE
	
7jzNONENONENONENONE
7jzNONENONENONENONE
	
v3zl9NONENONE
v3zl9NONENONE
	
p78ecNONENONE
p78ecNONENONE
	
7rhNONENONENONENONE
7rhNONENONENONENONE
	
exqoNONENONENONE
exqoNONE

'END TEST'